In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

/home/d3r/Documents/Github/vector_search_poc/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
es = Elasticsearch(
    ['http://localhost:9200'],
    http_auth=('kibana_user', 'Tamarinda192!'),
    verify_certs=False
)

/tmp/ipykernel_130335/2600257124.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [3]:
mapping = {
    'properties': {
        'embedding': {
            'type': 'dense_vector',
            'dims': 768  
        }
    }
}

In [6]:
es.indices.create(index='embeddings_test', body={'mappings': mapping})
docs = [
    {
        'title': 'Document 1',
        'text': 'This is the first document.'
    },
    {
        'title': 'Document 2',
        'text': 'This is the second document.'
    },
    {
        'title': 'Document 3',
        'text': 'This is the third document.'
    }
]

In [7]:
# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Generate embeddings for the documents using BERT
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
    doc['embedding'] = output.tolist()

# Index the documents in Elasticsearch
for doc in docs:
    es.index(index='embeddings_test', body=doc)
